In [2]:
import numpy
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px

In [79]:
def piksel(koord): #Odredjivanje piksel koordinata tacke
    return np.array([1,-1,-1])*(np.array([4032,0,0])-koord) #promeni 1600 za sliku

In [80]:
def fundJed(xl,xd): #Odredjivanje jednacina za fundamentalnu matricu
    return np.array([xl[0]*xd[0],xl[1]*xd[0],xl[2]*xd[0],xl[0]*xd[1],xl[1]*xd[1],xl[2]*xd[1],xl[0]*xd[2],xl[1]*xd[2],xl[2]*xd[2]])

In [81]:
def fundamentalnaMatrica(leve,desne): #Odredjivanje fundamentalne matrice f
    a=[]
    for x,xp in zip(leve,desne):
        jed= fundJed(x,xp)
        a.append(jed)
    a = np.array(a)
    U, D, V = np.linalg.svd(a)
    f = V[-1]
    f = f.reshape(3,3)
    return f

In [82]:
def popravka(f):
    U, S, V = np.linalg.svd(f)
    S = np.array([[S[0], 0, 0],
                  [0, S[1], 0],
                  [0, 0, 0]])
    FF1 = U@S@V
    if np.linalg.det(f) > np.linalg.det(FF1):
        return FF1
    return FF

In [83]:
def skew2v(EC): #Uzima vektor iz kososimetricne matrice 
    return np.array([EC[2][1],EC[0][2],EC[1][0]])

In [84]:
def dekomponuj(e): #Dekompozicija osnove matrice E
    q0=np.array([[0,-1,0],[1,0,0],[0,0,1]])
    e0=np.array([[0,1,0],[-1,0,0],[0,0,0]])
    U,S,V=np.linalg.svd(e)
    EC=U@e0@np.transpose(U)
    A=U@q0@V #probaj verziju bez transponovanih q0,v ako nesto ne valja
    return A,skew2v(EC) #skew2v mozda vrati suprotan znak, stavi mu minus ako ne valja

In [85]:
def kamera1(e): #Matrica kamere u koordinatnom sistemu druge kamere
    A,C=dekomponuj(e)
    C=K1 @ -np.transpose(A) @ C
    A=K1 @ np.transpose(A)
    A1=np.concatenate((np.transpose(A),[C]),axis=0) #append A.T,C
    return np.transpose(A1)

In [86]:
def triangulisi(T1,T2,pt1,pt2): #Triangulacija tacaka sa fotografije
    a=np.array([pt1[1]*T1[2,:]-pt1[2]*T1[1,:],
                -pt1[0]*T1[2,:]+pt1[2]*T1[0,:],
                pt2[1]*T2[2,:]-pt2[2]*T2[1,:],
                -pt2[0]*T2[2,:]+pt2[2]*T2[0,:]]) #ima na slajdovima
    U,S,V=np.linalg.svd(a)
    pt=V[3] # valjda moze -1
    pt=(1/pt[3])*pt
    pt=np.where(pt==0,0.0,pt)
    return np.array(pt)

In [87]:
def recPts(c1,c2,pts1,pts2):
    a=[]
    for x,xp in zip(pts1,pts2):
        X=triangulisi(c1,c2,x,xp)
        X=np.delete(X,-1)
        a.append(X)
    a = np.array(a)
    return a

In [112]:
K1=np.array([[2950,0,2016],[0,2950,954],[0,0,1]])
kam2=np.array([[2950,0,2016,0],
             [0,2316,954,0],
             [0,0,1,0]])
tacke1=np.array([[1058, 381, 1],
                [1686, 596, 1],
                [1108, 1007, 1],
                [424, 707, 1],
                [2780, 462, 1],
                [3346, 741, 1],
                [2513, 1211, 1], 
                [1935, 809, 1]])
tacke2=np.array([[1080, 301, 1],
                [1503, 592, 1],
                [639, 890, 1],
                [309, 502, 1],
                [3046, 666, 1],
                [3556, 1096, 1],
                [2180, 1409, 1],
                [1866, 875, 1]])
all1p=np.array([[1106, 889, 1],
                [1553, 1248, 1],
                [830,1597, 1],
                [506,1161, 1],
                [967,456, 1],
                [2761, 682, 1],
                [3266,992 ,1],
                [2451,1489,1],
                [1917,1063,1]])
all2p=np.array([[1176, 903, 1],
                [1694, 1217, 1],
                [1204,1688, 1],
                [603,1367, 1],
                [1074,545, 1],
                [3001, 940, 1],
                [3444,1390 ,1],
                [2152,1708,1],
                [1850,1123,1]])
allL=np.concatenate((tacke1[0:4],all1p),axis=0)
allD=np.concatenate((tacke2[0:4],all2p),axis=0)
allL=np.concatenate((allL,tacke1[4:]),axis=0)
allD=np.concatenate((allD,tacke2[4:]),axis=0)
#fundamentalna matrica
f=fundamentalnaMatrica(piksel(tacke1),piksel(tacke2))
f=popravka(f)
#osnovna matrica
e=np.transpose(K1)@f@K1
print("f:")
print(f)
print("e:")
print(e)
A,E0=dekomponuj(e)
print("A:")
print(A)
print("E0:")
print(E0)
print("T1:")
print(kamera1(e))
print("T2:")
print(kam2)
pts3D=recPts(kamera1(e),kam2,piksel(allL),piksel(allD))
pts3D

f:
[[ 1.64136303e-07 -7.52086881e-07  2.90249588e-04]
 [ 9.63007851e-07  5.25602575e-07 -1.38568768e-03]
 [-1.01888262e-03  1.89099533e-04  9.99998461e-01]]
e:
[[ 1.42839618 -6.54503608 -0.2842104 ]
 [ 8.38057582  4.57405641  3.11862496]
 [ 0.68064068 -2.43576415 -0.05936285]]
A:
[[-0.71992293 -0.28511604 -0.63278734]
 [ 0.37092179 -0.92865724 -0.00357154]
 [-0.58662424 -0.23728584  0.7743174 ]]
E0:
[ 0.35783798  0.01483866 -0.93366578]
T1:
[[-3.39947192e+03  1.08701906e+03 -1.69517634e+02  1.04205745e+03]
 [-1.44477145e+03 -2.74294611e+03  3.87055575e+01  5.93833795e+02]
 [-6.32787337e-01 -3.57153856e-03  7.74317397e-01  9.49441998e-01]]
T2:
[[2950    0 2016    0]
 [   0 2316  954    0]
 [   0    0    1    0]]


array([[-0.16244001,  0.1838065 , -0.54091919],
       [-0.07665043,  0.09629125, -0.4686147 ],
       [-0.24238189,  0.05521337, -0.5247099 ],
       [-0.3422543 ,  0.17256748, -0.60778986],
       [-0.15872825,  0.06323142, -0.57427889],
       [-0.0648388 , -0.02840533, -0.5702028 ],
       [-0.19022198, -0.08946153, -0.64607948],
       [-0.30280977,  0.01451568, -0.63709171],
       [-0.16849622,  0.15617263, -0.56953837],
       [ 0.17843351,  0.00737493, -0.53460165],
       [ 0.18415684, -0.07560002, -0.38794365],
       [ 0.01344155, -0.12962962, -0.43495905],
       [-0.02942179, -0.01385822, -0.49867472],
       [ 0.13239653,  0.04680076, -0.38366708],
       [ 0.22281462, -0.03138453, -0.43067786],
       [ 0.02191012, -0.0777124 , -0.39919988],
       [-0.02279911,  0.03461022, -0.47694986]])

In [109]:
import plotly.graph_objects as go
import plotly.express as px

# U ovom slucaju crtamo jednostavnu kocku. Vi cete koristiti rekonstruisana temena
temenaKocke = pts3D #np.array([[0,0, 3], [0, 3,3], [3,3,3],[3, 0,3],[0,0,0], [0, 3,0], [3, 3,0],[3,0,0]] )
# Ovo su ivice kocke zadate indeksima temena
ivice = [[0, 1], [0, 3], [0, 4], [2, 3], [2, 1], [2, 6], [5, 6], [5, 4], [5, 1], [7, 6],
[7, 3], [7, 4],
[9, 10], [9, 12], [9, 13], [11, 12], [11, 10], [11, 15], [14, 15],
[14, 13], [14, 10], [16, 15], [16, 12], [16,13]]

# dodatne mogucnosti istrazite sami

#  prikaz scene - koristi "plotly" biblioteku
def prikazKocke(): 
    # izdvajamo x,y,z koordinate svih tacaka
    xdata = (np.transpose(temenaKocke))[0]
    ydata = (np.transpose(temenaKocke))[1]
    zdata = (np.transpose(temenaKocke))[2]
    # u data1 ubacujemo sve sto treba naccrtati
    data1 = []
    # za svaku ivicu crtamo duz na osnovu koordinata
    for i in range(len(ivice)):
        data1.append(go.Scatter3d(x=[xdata[ivice[i][0]], xdata[ivice[i][1]]], y=[ydata[ivice[i][0]], ydata[ivice[i][1]]],z=[zdata[ivice[i][0]], zdata[ivice[i][1]]]))
    fig = go.Figure(data = data1 )
    # da ne prikazuje legendu
    fig.update_layout(showlegend=False)
    # pravi html fajl (ako zelite da napravite "rotatable" 3D rekonstruciju)
    # birate kao parametar velicinu apleta. fulhtml=False je vazno da ne bi pravio ogroman fajl
    # ovde stavite neki vas folder
    fig.write_html("./rekonstrukcija", include_plotlyjs = 'cdn', default_width = '800px', default_height = '600px', full_html = False) #Modifiy the html file
    fig.show()

prikazKocke()

In [76]:
def afinize(tacka):
    return list(map(lambda x : x / tacka[2], tacka))[:-1]
def cross(x, y):
    return np.cross(x, y)
def tacka_nedogleda(t1, t2, t3, t4):
    return list(cross(cross(t1, t2), cross(t3, t4)))
def osmoteme(lista_tacaka):
    t1, t2, t3, t4, t6, t7, t8 = lista_tacaka
    
    xb = tacka_nedogleda(t2, t6, t7, t3)
    yb = tacka_nedogleda(t2, t3, t4, t1)
    return afinize(tacka_nedogleda(t8, yb, t1, xb))

In [78]:
tacke_leve_kocke=np.concatenate((tacke1[0:4],all1p[1:4]),axis=0)
tacke_desne_kocke=np.concatenate((tacke2[0:4],all2p[1:4]),axis=0)
print(osmoteme(tacke_leve_kocke))
print(osmoteme(tacke_desne_kocke))

[437.4405777331684, 200.09008289255718]
[-22.07763721173984, 38.66060176035847]
